In [ ]:
import sys, os
print(f"🐍 Python Interpreter: {sys.executable}")
print(f"📂 Working Directory: {os.getcwd()}")
try:
    import numpy as np
    import biotite
    print(f"✅ Dependencies found: numpy {np.__version__}, biotite {biotite.__version__}")
except ImportError as e:
    print(f"❌ Missing Dependency: {e}")
    print("HINT: Ensure you are running in the correct virtual environment.")


# 🎓 The Virtual NMR Spectrometer

Welcome to the **Interactive Relaxation Lab**! 

This notebook simulates a **Nuclear Magnetic Resonance (NMR)** experiment on a virtual protein.
NMR relaxation rates ($R_1$, $R_2$, and Heteronuclear NOE) are the primary tools biophysicists use to study protein movement at the atomic level.

### 🎯 Goal
Understand how **molecular size** (tumbling rate) and **magnetic field strength** affect the signals we measure. This simulation uses the standard *Lipari-Szabo Model-Free* formalism, which is the gold standard for analyzing protein dynamics.

### ⚠️ How to Run (Important!)
This notebook uses advanced physics libraries that require a specific environment setup. Follow these steps strictly:

1.  **Run All Cells** (`Runtime` -> `Run all` or `Ctrl+F9`).
2.  **Wait for the Crash**: The first cell (`SETUP`) will install libraries and then **automatically crash/restart** the session. This is normal and required to load the correct components.
3.  **Wait 10 Seconds**: Allow the session to reconnect.
4.  **Run All Cells AGAIN**: This time, the setup will detect it is ready ('✅ Dependencies Ready') and proceed typically.

> **Note**: The step labeled *'Minimizing energy'* runs a full molecular dynamics simulation. On Google Colab (CPU), **this may take 5+ minutes**. Please be patient while it builds your custom protein structure!


In [ ]:
# 🛠️ SETUP: Install dependencies (Auto-Restart Mode)
# This cell automates the 'Factory Reset' to guarantee NumPy stability.

import os
import sys

# File marker to prevent infinite loops
if not os.path.exists("installed.marker"):
    try:
        import google.colab
        IN_COLAB = True
    except ImportError:
        IN_COLAB = False

    if IN_COLAB:
        print("🛡️ Locking Environment to NumPy 1.x...")
        
        # 1. Create constraint file (The only way to beat pip)
        with open("constraints.txt", "w") as f:
            f.write("numpy<2.0")
        
        # 2. Install with explicit constraints
        # We ignore 'shap' errors because we don't use it.
        !pip install -q "numpy<2.0" -c constraints.txt
        !pip install -q biotite -c constraints.txt
        !pip install -q openmm matplotlib ipywidgets py3Dmol -c constraints.txt
        !pip install -q --no-deps git+https://github.com/elkins/synth-pdb.git@master
        
        # 3. Create marker
        with open("installed.marker", "w") as f:
            f.write("done")
        
        print("🔄 Installation complete. KERNEL RESTARTING AUTOMATICALLY...")
        print("⚠️ Please wait 10 seconds, then Run All Cells again.")
        
        # 4. Kill the kernel to force reload of new C-extensions
        os.kill(os.getpid(), 9)
else:
    import numpy
    print(f"✅ Dependencies Ready. NumPy: {numpy.__version__}")


SyntaxError: unterminated string literal (detected at line 20) (1441161679.py, line 20)

In [ ]:
# ⚙️ HOTFIX: Patch synth_pdb for Colab compatibility (Physics + Generator + J-Coupling + Ions)
# This cell ensures the code handles missing OpenMM gracefully, preserves B-factors, AND adds J-Coupling/Ions.
import os
import sys
import synth_pdb
import base64
import importlib

try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    package_dir = os.path.dirname(synth_pdb.__file__)
    # Files to patch
    patches = {
        "physics.py": "...",
        "generator.py": "...",
        "j_coupling.py": "IiIiClNjYWxhciBDb3VwbGluZyAoSi1jb3VwbGluZykgY2FsY3VsYXRpb25zLgoKRURVQ0FUSU9OQUwgTk9URSAtIEthcnBsdXMgRXF1YXRpb246Cj09PT09PT09PT09PT09PT09PT09PT09PT09PT09PT09PT09PQpTY2FsYXIgY291cGxpbmdzIChKLWNvdXBsaW5ncykgYXJlIG1lZGlhdGVkIHRocm91Z2ggY2hlbWljYWwgYm9uZHMuClRoZSAzLWJvbmQgY291cGxpbmcgKF4zSikgZGVwZW5kcyBoZWF2aWx5IG9uIHRoZSB0b3JzaW9uIGFuZ2xlIGJldHdlZW4gdGhlIGF0b21zLgoKRm9yIHRoZSBiYWNrYm9uZSBhbWlkZSBwcm90b24gKEhOKSBhbmQgYWxwaGEgcHJvdG9uIChIQSksIHRoZSBjb3VwbGluZyBeM0pfSE5IYQp0ZWxscyB1cyBhYm91dCB0aGUgUGhpIGFuZ2xlLCBhbmQgdGh1cyB0aGUgc2Vjb25kYXJ5IHN0cnVjdHVyZS4KCkZvcm11bGE6CiAgXjNKID0gQSAqIGNvc14yKHRoZXRhKSArIEIgKiBjb3ModGhldGEpICsgQwoKV2hlcmUgdGhldGEgPSBQaGkgLSA2MCBkZWdyZWVzIChwaGFzZSBzaGlmdCkuClR5cGljYWwgdmFsdWVzOgotIEFscGhhIEhlbGl4IChQaGkgfiAtNjApOiB0aGV0YSB+IC0xMjAgLT4gSiBpcyBzbWFsbCAofjQgSHopCi0gQmV0YSBTaGVldCAoUGhpIH4gLTEyMCk6IHRoZXRhIH4gLTE4MCAtPiBKIGlzIGxhcmdlICh+OSBIeikKLSBSYW5kb20gQ29pbDogQXZlcmFnZWQgKH43IEh6KQoKVGhpcyBhbGxvd3MgTk1SIHNwZWN0cm9zY29waXN0cyB0byBkZXRlcm1pbmUgc2Vjb25kYXJ5IHN0cnVjdHVyZSBqdXN0IGJ5IG1lYXN1cmluZyBKLWNvdXBsaW5ncyEKIiIiCgppbXBvcnQgbnVtcHkgYXMgbnAKaW1wb3J0IGJpb3RpdGUuc3RydWN0dXJlIGFzIHN0cnVjCmltcG9ydCBsb2dnaW5nCmZyb20gdHlwaW5nIGltcG9ydCBEaWN0LCBMaXN0LCBUdXBsZQoKbG9nZ2VyID0gbG9nZ2luZy5nZXRMb2dnZXIoX19uYW1lX18pCgojIFZ1aXN0ZXIgJiBCYXggcGFyYW1ldGVycyAoSi4gQW0uIENoZW0uIFNvYy4gMTk5MywgMTE1LCA3NzcyLTc3NzcpCktBUlBMVVNfUEFSQU1TID0gewogICAgJ0EnOiA2LjUxLAogICAgJ0InOiAtMS43NiwKICAgICdDJzogMS42MAp9CgpkZWYgY2FsY3VsYXRlX2huX2hhX2NvdXBsaW5nKHN0cnVjdHVyZTogc3RydWMuQXRvbUFycmF5KSAtPiBEaWN0W3N0ciwgRGljdFtpbnQsIGZsb2F0XV06CiAgICAiIiIKICAgIENhbGN1bGF0ZSAzSl9ITkhhIGNvdXBsaW5nIGNvbnN0YW50cyBmb3IgdGhlIHByb3RlaW4gYmFja2JvbmUuCiAgICAKICAgIEFyZ3M6CiAgICAgICAgc3RydWN0dXJlOiBBdG9tQXJyYXkgY29udGFpbmluZyB0aGUgcHJvdGVpbgogICAgICAgIAogICAgUmV0dXJuczoKICAgICAgICBEaWN0IGtleWVkIGJ5IENoYWluIElEIC0+IFJlc2lkdWUgSUQgLT4gSi1jb3VwbGluZyB2YWx1ZSAoSHopCiAgICAiIiIKICAgIGxvZ2dlci5pbmZvKCJDYWxjdWxhdGluZyAzSl9ITkhhIHNjYWxhciBjb3VwbGluZ3MuLi4iKQogICAgCiAgICBwaGksIHBzaSwgb21lZ2EgPSBzdHJ1Yy5kaWhlZHJhbF9iYWNrYm9uZShzdHJ1Y3R1cmUpCiAgICAKICAgICMgYmlvdGl0ZSByZXR1cm5zIGFuZ2xlcyBmb3IgZWFjaCByZXNpZHVlLgogICAgIyBUaGUgZmlyc3QgcmVzaWR1ZSBoYXMgbm8gUGhpICh1bmRlZmluZWQpLgogICAgIyBUaGUgY29ycmVzcG9uZGluZyByZXNpZHVlcyBhcmUgc3RydWN0dXJlIHJlc2lkdWVzIHRoYXQgaGF2ZSBiYWNrYm9uZSBhdG9tcy4KICAgICMgV2UgbmVlZCB0byBtYXAgdGhlc2UgYmFjayB0byBSZXMgSURzLgogICAgCiAgICByZXNfc3RhcnRzID0gc3RydWMuZ2V0X3Jlc2lkdWVfc3RhcnRzKHN0cnVjdHVyZSkKICAgICMgRmlsdGVyIHRvIG9ubHkgYW1pbm8gYWNpZHM/IFVzdWFsbHkgc2FmZS4KICAgIAogICAgcmVzdWx0cyA9IHt9CiAgICAKICAgICMgSXRlcmF0ZSBvdmVyIHJlc2lkdWVzCiAgICAjIEFuZ2xlcyBhcnJheSBtYXRjaGVzIG51bWJlciBvZiByZXNpZHVlcwogICAgaWYgbGVuKHBoaSkgIT0gbGVuKHJlc19zdGFydHMpOgogICAgICAgIGxvZ2dlci53YXJuaW5nKGYiTWlzbWF0Y2ggaW4gYmFja2JvbmUgYW5nbGVzIGNvdW50ICh7bGVuKHBoaSl9KSB2cyByZXNpZHVlIGNvdW50ICh7bGVuKHJlc19zdGFydHMpfSkuIikKICAgICAgICByZXR1cm4ge30KICAgICAgICAKICAgIGZvciBpLCBzdGFydF9pZHggaW4gZW51bWVyYXRlKHJlc19zdGFydHMpOgogICAgICAgICMgR2V0IHJlc2lkdWUgaW5mbwogICAgICAgIHJlc19hdG9tcyA9IHN0cnVjdHVyZVtzdGFydF9pZHggOiByZXNfc3RhcnRzW2krMV0gaWYgaSsxIDwgbGVuKHJlc19zdGFydHMpIGVsc2UgTm9uZV0KICAgICAgICBjaGFpbl9pZCA9IHJlc19hdG9tcy5jaGFpbl9pZFswXQogICAgICAgIHJlc19pZCA9IHJlc19hdG9tcy5yZXNfaWRbMF0KICAgICAgICByZXNfbmFtZSA9IHJlc19hdG9tcy5yZXNfbmFtZVswXQogICAgICAgIAogICAgICAgIGlmIGNoYWluX2lkIG5vdCBpbiByZXN1bHRzOgogICAgICAgICAgICByZXN1bHRzW2NoYWluX2lkXSA9IHt9CiAgICAgICAgICAgIAogICAgICAgICMgR2V0IFBoaSBhbmdsZSAoaW4gcmFkaWFucykKICAgICAgICBwaGlfcmFkID0gcGhpW2ldCiAgICAgICAgCiAgICAgICAgIyBDaGVjayBmb3IgTmFOICh1bmRlZmluZWQsIGUuZy4gTi10ZXJtaW51cykKICAgICAgICBpZiBucC5pc25hbihwaGlfcmFkKToKICAgICAgICAgICAgIyBObyBjb3VwbGluZyBkZWZpbmVkCiAgICAgICAgICAgIGNvbnRpbnVlCiAgICAgICAgICAgIAogICAgICAgICMgR2x5Y2luZSBoYXMgSEEyL0hBMywgdXN1YWxseSBhdmVyYWdlZCBvciBzcGVjaWZpYy4KICAgICAgICAjIFRoaXMgZXF1YXRpb24gYXNzdW1lcyBzdGFuZGFyZCBILU4tQ2EtSGEgZ2VvbWV0cnkuCiAgICAgICAgIyBTdGFuZGFyZCBLYXJwbHVzOiB0aGV0YSA9IHBoaSAtIDYwIGRlZyAoLSBwaS8zKQogICAgICAgIHRoZXRhID0gcGhpX3JhZCAtIChucC5kZWcycmFkKDYwLjApKQogICAgICAgIAogICAgICAgICMgQ2FsY3VsYXRlIEoKICAgICAgICAjIEogPSBBIGNvc14yKHRoZXRhKSArIEIgY29zKHRoZXRhKSArIEMKICAgICAgICBjb3NfdGhldGEgPSBucC5jb3ModGhldGEpCiAgICAgICAgal92YWwgPSAoS0FSUExVU19QQVJBTVNbJ0EnXSAqIChjb3NfdGhldGEgKiogMikpICsgXAogICAgICAgICAgICAgICAgKEtBUlBMVVNfUEFSQU1TWydCJ10gKiBjb3NfdGhldGEpICsgXAogICAgICAgICAgICAgICAgS0FSUExVU19QQVJBTVNbJ0MnXQogICAgICAgICAgICAgICAgCiAgICAgICAgIyBBZGQgbm9pc2U/IE9wdGlvbmFsLiBSZWFsIG1lYXN1cmVtZW50cyBoYXZlIGVycm9yIH4wLjUgSHouCiAgICAgICAgIyBGb3IgcHVyZSBlZHVjYXRpb24sIGNsZWFuIGN1cnZlcyBhcmUgYmV0dGVyLgogICAgICAgIAogICAgICAgIHJlc3VsdHNbY2hhaW5faWRdW3Jlc19pZF0gPSByb3VuZChqX3ZhbCwgMikKICAgICAgICAKICAgIHJldHVybiByZXN1bHRzCg==",
        "cofactors.py": "CiIiIgpDb2ZhY3RvciBhbmQgTWV0YWwgSW9uIENvb3JkaW5hdGlvbiBNb2R1bGUuCgpUSEUgIkFJIFRSSU5JVFkiIFBIQVNFIDE1OiBJbm9yZ2FuaWMgQ29vcmRpbmF0aW9uLgoKQmlvbG9naWNhbCBwcm90ZWlucyBhcmVuJ3QganVzdCBjaGFpbnMgb2YgYW1pbm8gYWNpZHM7IHRoZXkgb2Z0ZW4gcmVxdWlyZSAKaW5vcmdhbmljICJjb2ZhY3RvcnMiIG9yIG1ldGFsIGlvbnMgdG8gZnVuY3Rpb24uIFppbmMgKFpuMispIGlzIG9uZSBvZiAKdGhlIG1vc3QgY29tbW9uLCBmb3VuZCBpbiAiWmluYyBGaW5nZXIiIG1vdGlmcyB3aGVyZSBpdCBzdGFiaWxpemVzIHRoZSAKZm9sZCBieSBjb29yZGluYXRpbmcgd2l0aCBDeXN0ZWluZSAoQykgYW5kIEhpc3RpZGluZSAoSCkgcmVzaWR1ZXMuCgpFZHVjYXRpb25hbCBOb3RlIC0gQ29vcmRpbmF0aW9uIENoZW1pc3RyeToKLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tCjEuIExpZ2FuZHM6IFRoZSBhdG9tcyB0aGF0IGRvbmF0ZSBlbGVjdHJvbnMgdG8gdGhlIG1ldGFsIChlLmcuLCBDeXMgU3VsZnVyLCBIaXMgTml0cm9nZW4pLgoyLiBDb29yZGluYXRpb24gTnVtYmVyOiBUaGUgbnVtYmVyIG9mIGxpZ2FuZHMgKFppbmMgaXMgdXN1YWxseSA0IC0gVGV0cmFoZWRyYWwpLgozLiBHZW9tZXRyaWMgQ2VudHJvaWQ6IFRoZSBpZGVhbCBwb3NpdGlvbiBmb3IgdGhlIG1ldGFsIGlzIHRoZSBjZW50ZXIgb2YgaXRzIGxpZ2FuZHMuCgpUaGlzIG1vZHVsZSBhdXRvbWF0aWNhbGx5IGRldGVjdHMgdGhlc2UgbW90aWZzIGFuZCBpbnNlcnRzIHRoZSBhcHByb3ByaWF0ZSBpb25zLgoiIiIKCmltcG9ydCBudW1weSBhcyBucAppbXBvcnQgYmlvdGl0ZS5zdHJ1Y3R1cmUgYXMgc3RydWMKaW1wb3J0IGxvZ2dpbmcKZnJvbSB0eXBpbmcgaW1wb3J0IExpc3QsIERpY3QKCmxvZ2dlciA9IGxvZ2dpbmcuZ2V0TG9nZ2VyKF9fbmFtZV9fKQoKZGVmIGZpbmRfbWV0YWxfYmluZGluZ19zaXRlcyhzdHJ1Y3R1cmU6IHN0cnVjLkF0b21BcnJheSwgZGlzdGFuY2VfdGhyZXNob2xkOiBmbG9hdCA9IDEwLjApIC0+IExpc3RbRGljdF06CiAgICAiIiIKICAgIFNjYW5zIHRoZSBzdHJ1Y3R1cmUgZm9yIGNsdXN0ZXJzIG9mIHJlc2lkdWVzIHRoYXQgY291bGQgY29vcmRpbmF0ZSBhIG1ldGFsIGlvbi4KICAgIAogICAgQXJnczoKICAgICAgICBzdHJ1Y3R1cmU6IEJpb3RpdGUgQXRvbUFycmF5LgogICAgICAgIGRpc3RhbmNlX3RocmVzaG9sZDogTWF4IGRpc3RhbmNlIGJldHdlZW4gYW55IHR3byBjb29yZGluYXRpbmcgYXRvbXMgaW4gYSBjbHVzdGVyLgogICAgICAgIAogICAgUmV0dXJuczoKICAgICAgICBMaXN0W0RpY3RdOiBBIGxpc3Qgb2YgZGV0ZWN0ZWQgc2l0ZXMsIGVhY2ggd2l0aCAndHlwZScgYW5kICdsaWdhbmRfaW5kaWNlcycuCiAgICAiIiIKICAgIGxvZ2dlci5pbmZvKCJTY2FubmluZyBmb3IgbWV0YWwgYmluZGluZyBtb3RpZnMgKFppbmMgRmluZ2VycykuLi4iKQogICAgCiAgICAjIFN0YW5kYXJkIGxpZ2FuZHMgZm9yIFppbmMKICAgICMgQ1lTIChTRyksIEhJUyAoTkUyIG9yIE5EMSkgLSBpbmNsdWRpbmcgSElEL0hJRS9ISVAgdmFyaWFudHMKICAgIGxpZ2FuZF9tYXNrID0gKAogICAgICAgICgoc3RydWN0dXJlLnJlc19uYW1lID09ICJDWVMiKSAmIChzdHJ1Y3R1cmUuYXRvbV9uYW1lID09ICJTRyIpKSB8CiAgICAgICAgKChucC5pc2luKHN0cnVjdHVyZS5yZXNfbmFtZSwgWyJISVMiLCAiSElEIiwgIkhJRSIsICJISVAiXSkpICYgKChzdHJ1Y3R1cmUuYXRvbV9uYW1lID09ICJORTIiKSB8IChzdHJ1Y3R1cmUuYXRvbV9uYW1lID09ICJORDEiKSkpCiAgICApCiAgICAKICAgIGNhbmRpZGF0ZV9pbmRpY2VzID0gbnAud2hlcmUobGlnYW5kX21hc2spWzBdCiAgICBpZiBsZW4oY2FuZGlkYXRlX2luZGljZXMpIDwgMzoKICAgICAgICByZXR1cm4gW10KICAgICAgICAKICAgIGNhbmRpZGF0ZV9jb29yZHMgPSBzdHJ1Y3R1cmUuY29vcmRbY2FuZGlkYXRlX2luZGljZXNdCiAgICBzaXRlcyA9IFtdCiAgICBhc3NpZ25lZF9pbmRpY2VzID0gc2V0KCkKICAgIAogICAgZm9yIGkgaW4gcmFuZ2UobGVuKGNhbmRpZGF0ZV9pbmRpY2VzKSk6CiAgICAgICAgaWR4X2kgPSBjYW5kaWRhdGVfaW5kaWNlc1tpXQogICAgICAgIGlmIGlkeF9pIGluIGFzc2lnbmVkX2luZGljZXM6CiAgICAgICAgICAgIGNvbnRpbnVlCiAgICAgICAgICAgIAogICAgICAgICMgRmluZCBhbGwgYXZhaWxhYmxlIG5laWdoYm9ycyB3aXRoaW4gdGhyZXNob2xkCiAgICAgICAgZGlmZnMgPSBjYW5kaWRhdGVfY29vcmRzIC0gY2FuZGlkYXRlX2Nvb3Jkc1tpXQogICAgICAgIGRpc3RzID0gbnAuc3FydChucC5zdW0oZGlmZnMqKjIsIGF4aXM9LTEpKQogICAgICAgIAogICAgICAgIG5laWdoYm9yX21hc2sgPSBkaXN0cyA8IGRpc3RhbmNlX3RocmVzaG9sZAogICAgICAgICMgT25seSB0YWtlIG5laWdoYm9ycyB0aGF0IGFyZW4ndCBhbHJlYWR5IGFzc2lnbmVkCiAgICAgICAgdW5hc3NpZ25lZF9uZWlnaGJvcnMgPSBbCiAgICAgICAgICAgIGNhbmRpZGF0ZV9pbmRpY2VzW2pdIGZvciBqIGluIHJhbmdlKGxlbihjYW5kaWRhdGVfaW5kaWNlcykpIAogICAgICAgICAgICBpZiBuZWlnaGJvcl9tYXNrW2pdIGFuZCBjYW5kaWRhdGVfaW5kaWNlc1tqXSBub3QgaW4gYXNzaWduZWRfaW5kaWNlcwogICAgICAgIF0KICAgICAgICAKICAgICAgICAjIFppbmMgZmluZ2VycyB0eXBpY2FsbHkgaGF2ZSA0IGxpZ2FuZHMgKEM0LCBDMkgyKQogICAgICAgICMgV2UgZ3JlZWRpbHkgdGFrZSB0aGUgbGFyZ2VzdCBjbHVzdGVyIGZpcnN0CiAgICAgICAgaWYgbGVuKHVuYXNzaWduZWRfbmVpZ2hib3JzKSA+PSA0OgogICAgICAgICAgICAjIFNvcnQgYnkgZGlzdGFuY2UgdG8gY2VudGVyIGlmIHdlIGhhZCBtb3JlIHRoYW4gNCwgYnV0IDQgaXMgdGhlIHN3ZWV0IHNwb3QKICAgICAgICAgICAgY2x1c3RlciA9IHVuYXNzaWduZWRfbmVpZ2hib3JzWzo0XQogICAgICAgICAgICBzaXRlcy5hcHBlbmQoewogICAgICAgICAgICAgICAgInR5cGUiOiAiWk4iLAogICAgICAgICAgICAgICAgImxpZ2FuZF9pbmRpY2VzIjogY2x1c3RlcgogICAgICAgICAgICB9KQogICAgICAgICAgICBmb3IgaWR4IGluIGNsdXN0ZXI6CiAgICAgICAgICAgICAgICBhc3NpZ25lZF9pbmRpY2VzLmFkZChpZHgpCiAgICAgICAgICAgICAgICAKICAgIGlmIHNpdGVzOgogICAgICAgIGxvZ2dlci5pbmZvKGYiRm91bmQge2xlbihzaXRlcyl9IHBvdGVudGlhbCBtZXRhbCBiaW5kaW5nIHNpdGVzLiIpCiAgICByZXR1cm4gc2l0ZXMKCmRlZiBhZGRfbWV0YWxfaW9uKHN0cnVjdHVyZTogc3RydWMuQXRvbUFycmF5LCBzaXRlOiBEaWN0KSAtPiBzdHJ1Yy5BdG9tQXJyYXk6CiAgICAiIiIKICAgIEFkZHMgYSBtZXRhbCBpb24gKEhFVEFUTSkgdG8gdGhlIHN0cnVjdHVyZSBhdCB0aGUgY2VudHJvaWQgb2YgaXRzIGxpZ2FuZHMuCiAgICAKICAgIEFyZ3M6CiAgICAgICAgc3RydWN0dXJlOiBPcmlnaW5hbCBBdG9tQXJyYXkuCiAgICAgICAgc2l0ZTogSWRlbnRpZmljYXRpb24gb2YgdGhlIHNpdGUgKGZyb20gZmluZF9tZXRhbF9iaW5kaW5nX3NpdGVzKS4KICAgICAgICAKICAgIFJldHVybnM6CiAgICAgICAgc3RydWMuQXRvbUFycmF5OiBOZXcgQXRvbUFycmF5IHdpdGggdGhlIGlvbiBhcHBlbmRlZC4KICAgICIiIgogICAgaW9uX3R5cGUgPSBzaXRlWyJ0eXBlIl0KICAgIGxpZ2FuZF9pbmRpY2VzID0gc2l0ZVsibGlnYW5kX2luZGljZXMiXQogICAgCiAgICAjIENhbGN1bGF0ZSBDZW50cm9pZAogICAgbGlnYW5kX2Nvb3JkcyA9IHN0cnVjdHVyZS5jb29yZFtsaWdhbmRfaW5kaWNlc10KICAgIGNlbnRyb2lkID0gbnAubWVhbihsaWdhbmRfY29vcmRzLCBheGlzPTApCiAgICAKICAgICMgQ3JlYXRlIHRoZSBJb24gQXRvbQogICAgaW9uID0gc3RydWMuQXRvbUFycmF5KDEpCiAgICBpb24ucmVzX25hbWUgPSBucC5hcnJheShbaW9uX3R5cGVdKQogICAgaW9uLmF0b21fbmFtZSA9IG5wLmFycmF5KFtpb25fdHlwZV0pCiAgICBpb24uZWxlbWVudCA9IG5wLmFycmF5KFtpb25fdHlwZVs6MV1dKSAjICJaIiBmb3IgIlpOIgogICAgaW9uLmNvb3JkID0gbnAuYXJyYXkoW2NlbnRyb2lkXSkKICAgIAogICAgIyBNZXRhZGF0YQogICAgIyBQaWNrIGEgcmVzaWR1ZSBJRCBoaWdoZXIgdGhhbiBleGlzdGluZwogICAgbWF4X3Jlc19pZCA9IG5wLm1heChzdHJ1Y3R1cmUucmVzX2lkKQogICAgaW9uLnJlc19pZCA9IG5wLmFycmF5KFttYXhfcmVzX2lkICsgMV0pCiAgICBpb24uY2hhaW5faWQgPSBucC5hcnJheShbc3RydWN0dXJlLmNoYWluX2lkWzBdXSkKICAgIGlvbi5oZXRlcm8gPSBucC5hcnJheShbVHJ1ZV0pCiAgICAKICAgIGxvZ2dlci5pbmZvKGYiSW5qZWN0ZWQge2lvbl90eXBlfSBpb24gYXQgY29vcmRpbmF0ZWQgc2l0ZSB7Y2VudHJvaWR9LiIpCiAgICAKICAgIHJldHVybiBzdHJ1Y3R1cmUgKyBpb24K"
    }

    print(f"🚨 Applying robust hotfix to {len(patches)} files in: {package_dir}")

    for file_name, encoded_content in patches.items():
        target_path = os.path.join(package_dir, file_name)
        with open(target_path, "w") as f:
            f.write(base64.b64decode(encoded_content).decode("utf-8"))

    print("✅ Hotfixes applied: OpenMM optional + B-factors preserved + J-Couplings + Metal Ions added.")
    
    # Force reload modules to pick up changes
    importlib.reload(synth_pdb)
    for mod_name in ["synth_pdb.physics", "synth_pdb.generator", "synth_pdb.j_coupling", "synth_pdb.cofactors"]:
        try:
            if mod_name in sys.modules:
                importlib.reload(sys.modules[mod_name])
        except Exception:
            pass
else:
    print("✅ Running locally. Skipping hotfixes to preserve local source code.")


In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
import numpy as np
from synth_pdb.generator import generate_pdb_content
from synth_pdb.relaxation import calculate_relaxation_rates, predict_order_parameters
import biotite.structure.io.pdb as pdb
import io

# 1. Generate a Test Protein (Zinc Finger-like Motif)
# Visualization: Beta Hairpin (Sheet-Turn-Sheet) + Alpha Helix
# This provides a rich mix of rigid and flexible regions.
sequence = "VKITVGGTLTVALGGALALALALALALAA"
structure_def = "1-5:beta,6-8:random,9-13:beta,14-16:random,17-29:alpha"

print("🧬 Generating virtual protein (Zinc Finger-like fold)...")
print("   - Optimizing side-chains (Monte Carlo)...")
print("   - Minimizing energy (OpenMM if available)...")

pdb_content = generate_pdb_content(
    sequence_str=sequence, 
    structure=structure_def, 
    optimize_sidechains=True, 
    minimize_energy=True
)
pdb_file = pdb.PDBFile.read(io.StringIO(pdb_content))
structure = pdb_file.get_structure(model=1)

# Pre-calculate Order Parameters (S2) based on structure
s2_map = predict_order_parameters(structure)
res_ids = sorted(list(s2_map.keys()))
s2_values = [s2_map[r] for r in res_ids]

print("✅ Protein Model Ready!")


### 🧬 The Simulated Protein (Zinc Finger Motif)

We have generated a synthetic **Zinc Finger-like** fold to demonstrate contrast between different secondary structures:
*   **Residues 1-5**: Beta Strand 1 (Rigid, Extended)
*   **Residues 6-8**: Turn (Flexible)
*   **Residues 9-13**: Beta Strand 2 (Rigid, Extended)
*   **Residues 14-16**: Loop (Flexible)
*   **Residues 17-29**: Alpha Helix (Rigid, Compact)

Notice how the physics engine (`synth-pdb`) automatically assigns different **Order Parameters ($S^2$)** to these regions. $S^2$ represents spatial restriction: **1.0** is completely rigid, **0.0** is completely disordered.


In [ ]:
# Visualizing the Structure
import py3Dmol
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

def pdb_to_bfactor_map(pdb_str):
    """Extract average B-factor per residue."""
    res_b = {}
    counts = {}
    for line in pdb_str.splitlines():
        if line.startswith("ATOM"):
            try:
                res_id = int(line[22:26].strip())
                b = float(line[60:66].strip())
                if res_id not in res_b:
                    res_b[res_id] = 0.0
                    counts[res_id] = 0
                res_b[res_id] += b
                counts[res_id] += 1
            except ValueError:
                pass
    # Average
    for r in res_b:
        if counts[r] > 0:
            res_b[r] /= counts[r]
    return res_b

res_b_map = pdb_to_bfactor_map(pdb_content)
if not res_b_map:
    res_b_map = {i: 50 for i in range(1, 30)}

vals = list(res_b_map.values())
min_val = min(vals)
max_val = max(vals)

print(f"📏 Data Stats: Min={min_val:.1f}, Max={max_val:.1f}, Mean={np.mean(vals):.1f}")

# Force Physics Range Scaling
# Prevents one outlier (e.g. 100) from squashing all 20s to Blue.
vmin = 15.0  # Rigid Limit
vmax = 65.0  # Flexible Limit

# Setup Viewer
view = py3Dmol.view(width=400, height=300)
view.addModel(pdb_content, 'pdb')

# Apply Logic: Python-calculated Colors
try:
    cmap = matplotlib.colormaps['bwr']
except AttributeError:
    cmap = plt.get_cmap('bwr')

# Clip values to range for color lookup
norm = mcolors.Normalize(vmin=vmin, vmax=vmax)

view.setStyle({'cartoon': {'color': 'white'}}) # Base

for r, b_val in res_b_map.items():
    rgba = cmap(norm(b_val))
    hex_color = mcolors.to_hex(rgba)
    view.addStyle({'resi': r}, {'cartoon': {'color': hex_color}})

view.zoomTo()
print("\n--- COLOR GUIDE ---")
print("🔵 BLUE (Cool) = RIGID (Alpha Helices / Beta Sheets)")
print("🔴 RED  (Hot)  = FLEXIBLE (Loops / Termini)")
print("-------------------")
view.show()


### 📊 Guide to the Plots

When you run the simulator below, you will see three coupled plots. Here is how to interpret them:

1.  **$R_1$ (Longitudinal Rate - Blue)**: 
    *   *Physics*: Sensitive to fast interaction fluctuations (nanosecond scale).
    *   *Pattern*: Often relatively flat for folded proteins, but may dip in flexible loops.

2.  **$R_2$ (Transverse Rate - Red)**: 
    *   *Physics*: Sensitive to slow motions (global tumbling) and chemical exchange.
    *   *Key Insight*: **$R_2$ scales with protein size**. Large proteins (slow tumbling) have high $R_2$ rates. High $R_2$ means the signal decays fast (broad lines), making large proteins hard to study!
    *   *Flexible Regions*: $R_2$ drops sharply because local flexibility averages out the magnetic interactions.

3.  **Heteronuclear NOE (Green)**: 
    *   *The Rigidity Sensor*: This is the most robust indicator of local structure.
    *   **Value ~ 0.8**: Rigid backbone (Helix/Sheet).
    *   **Value < 0.6**: Flexible loop/terminus.
    *   **Negative Values**: Extremely flexible or unfolded.

---


In [ ]:
# Interactive Relaxation Dynamics Exploration
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import io

# 1. Create the persistent Image widget
# We will update this widget's value, rather than printing to the log.
plot_image = widgets.Image(format='png', width=800, height=1000)

# 2. Create Sliders
field_slider = widgets.IntSlider(min=400, max=1200, step=100, value=600, description='Field (MHz)')
tumbling_slider = widgets.FloatSlider(min=2.0, max=50.0, step=1.0, value=10.0, description='Tumbling (ns)')

# 3. Update Function
def update_relaxation_plot(change=None):
    field_mhz = field_slider.value
    tau_m_ns = tumbling_slider.value
    
    # Stop Matplotlib from trying to be helpful
    plt.ioff()
    
    # Calculate
    rates = calculate_relaxation_rates(
        structure, 
        field_mhz=field_mhz, 
        tau_m_ns=tau_m_ns, 
        s2_map=s2_map
    )
    r1 = [rates[r]['R1'] for r in res_ids]
    r2 = [rates[r]['R2'] for r in res_ids]
    noe = [rates[r]['NOE'] for r in res_ids]
    
    # Plot to Memory
    # Note: We create a new figure each time but NEVER display it directly.
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 12), sharex=True)
    
    ax1.plot(res_ids, r1, 'o-', color='blue', label='R1 (Longitudinal)')
    ax1.set_ylabel('$R_1$ ($s^{-1}$)')
    ax1.set_title(f'Relaxation Dynamics @ {field_mhz} MHz, $\\tau_m$={tau_m_ns}ns')
    ax1.grid(True, alpha=0.3)
    
    ax2.plot(res_ids, r2, 's-', color='red', label='R2 (Transverse)')
    ax2.set_ylabel('$R_2$ ($s^{-1}$)')
    ax2.grid(True, alpha=0.3)
    
    ax3.plot(res_ids, noe, '^-', color='green', label='HetNOE')
    ax3.set_ylabel('NOE Ratio')
    ax3.set_xlabel('Residue Number')
    ax3.grid(True, alpha=0.3)
    
    for r in res_ids:
        if s2_map[r] < 0.75:
            for ax in [ax1, ax2, ax3]:
                ax.axvspan(r-0.5, r+0.5, color='yellow', alpha=0.1)
    
    plt.tight_layout()
    
    # Save to buffer
    buf = io.BytesIO()
    fig.savefig(buf, format='png')
    plt.close(fig) # Essential: kill the object
    
    # Update Image Widget
    buf.seek(0)
    plot_image.value = buf.read()

# 4. Bind Events
# continuous_update=False prevents lag/stacking from rapid drags
field_slider.continuous_update = False
tumbling_slider.continuous_update = False

field_slider.observe(update_relaxation_plot, names='value')
tumbling_slider.observe(update_relaxation_plot, names='value')

# 5. Initial Render
update_relaxation_plot()

# 6. Layout UI
ui = widgets.VBox([field_slider, tumbling_slider])
display(ui, plot_image)


### 🧪 Try These Experiments

Use the sliders above to change the "experimental" conditions:

1.  **Simulate a Giant Protein Complex**:
    *   Slide **Tumbling (ns)** to **40.0 ns**.
    *   *Observation*: Look at the **$R_2$ (Red)** plot. It shoots up significantly! This is why NMR is limited to smaller proteins (< 50-100 kDa) without special tricks (like TROSY).
    *   *Note*: The NOE profile stays mostly the same—local rigidity hasn't changed, only the global tumbling.

2.  **Go to High Field**:
    *   Slide **Field (MHz)** from **600** to **1200**.
    *   *Observation*: $R_2$ increases. This is due to **Chemical Shift Anisotropy (CSA)** becoming a stronger relaxation mechanism at high magnetic fields.


# 🔬 Advanced Spectroscopy: Measuring Geometry Directly

Relaxation tells us about **Motion**. J-Couplings and Chemical Shifts tell us about **Geometry**.

Here we simulate two classic experiments:
1.  **J-Coupling ($^3J_{HNH\alpha}$)**: Measures the torsion angle ($\phi$) of the backbone.
2.  **HSQC Spectrum**: The 2D "fingerprint" of the protein.


In [ ]:
# Calculate J-Couplings using the Karplus Equation
from synth_pdb.j_coupling import calculate_hn_ha_coupling

# 1. Calculate
j_couplings = calculate_hn_ha_coupling(structure)

# 2. Extract Data
res_nums = []
j_vals = []
colors = []

for r in res_ids:
    if r in j_couplings.get('A', {}):
        val = j_couplings['A'][r]
        res_nums.append(r)
        j_vals.append(val)
        # Color by physics expectation
        if val < 6.0: colors.append('blue') # Helix-like
        elif val > 8.0: colors.append('red') # Sheet-like
        else: colors.append('gray') # Random/Avg

# 3. Plot
plt.figure(figsize=(10, 4))
plt.bar(res_nums, j_vals, color=colors, alpha=0.7, edgecolor='black')
plt.axhline(4.0, color='blue', linestyle='--', alpha=0.5, label='Alpha Helix (~4 Hz)')
plt.axhline(9.0, color='red', linestyle='--', alpha=0.5, label='Beta Sheet (~9 Hz)')
plt.xlabel('Residue Number')
plt.ylabel('$^3J_{HNH\\alpha}$ (Hz)')
plt.title('Backbone Dihedral Probe: Scalar Couplings')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("📏 INTERPRETATION:")
print("Smaller couplings (< 6 Hz) indicate Alpha-Helical turns.")
print("Larger couplings (> 8 Hz) indicate Extended/Beta conformations.")


In [ ]:
# Simulate 2D HSQC Spectrum
from synth_pdb.chemical_shifts import predict_chemical_shifts

# 1. Predict Shifts (SPARTA-lite)
shifts = predict_chemical_shifts(structure)

# 2. Collect (H, N) pairs
h_shifts = []
n_shifts = []
labels = []
ss_colors = []

from synth_pdb.structure_utils import get_secondary_structure
ss_types = get_secondary_structure(structure)

for i, r in enumerate(res_ids):
    chain_shifts = shifts.get('A', {})
    res_shifts = chain_shifts.get(r, {})
    
    if 'H' in res_shifts and 'N' in res_shifts:
        h = res_shifts['H']
        n = res_shifts['N']
        h_shifts.append(h)
        n_shifts.append(n)
        labels.append(f"{structure[structure.res_id==r].res_name[0]}{r}")
        
        # Color by SS
        ss = ss_types[i] if i < len(ss_types) else 'coil'
        if ss == 'alpha': ss_colors.append('blue')
        elif ss == 'beta': ss_colors.append('red')
        else: ss_colors.append('gray')

# 3. Plot HSQC
plt.figure(figsize=(8, 8))
plt.scatter(h_shifts, n_shifts, c=ss_colors, s=100, alpha=0.8, edgecolors='black')

# Standard NMR axes orientation (High -> Low)
plt.xlim(10.5, 6.5)  # Proton: 10 down to 6
plt.ylim(135, 100)   # Nitrogen: 135 down to 100

plt.xlabel('$^{1}$H Chemical Shift (ppm)')
plt.ylabel('$^{15}$N Chemical Shift (ppm)')
plt.title('Synthetic 2D $^{1}$H-$^{15}$N HSQC Spectrum')

# Label peaks
for h, n, txt in zip(h_shifts, n_shifts, labels):
    plt.annotate(txt, (h, n), xytext=(3, 3), textcoords='offset points', fontsize=8)

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("📊 INTERPRETATION:")
print("Each dot represents one residue's backbone amide group.")
print("Dispersion (spread) indicates a folded structure.")
print("Collapsed peaks in the center would indicate an unfolded/disordered protein.")


### 🗺️ 3. The Contact Map (Virtual NOESY)

The **Contact Map** is the "Fingerprint" of a protein fold. It shows which residues are touching in 3D space.

*   **Diagonal**: Residues are always close to their neighbors (i, i+1).
*   **Off-Diagonal**: These are the interesting **Long-Range Contacts**. They tell us the protein has folded back on itself (e.g., a Beta Sheet or Alpha Helix packing).
*   **AI Relevance**: This 2D matrix is exactly what tools like **AlphaFold** predict internally (the "Distogram").


In [ ]:
# Calculate Contact Map
from synth_pdb.contact import compute_contact_map

# Compute Alpha-Carbon distances (Standard for Fold Recognition)
contact_map = compute_contact_map(structure, method='ca', power=None)
binary_map = compute_contact_map(structure, method='ca', threshold=8.0, power=0)

# Plotting
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# 1. Distance Matrix (Continuous)
im1 = ax1.imshow(contact_map, cmap='viridis_r', interpolation='nearest')
ax1.set_title('Distance Matrix (Angstroms)')
ax1.set_xlabel('Residue Index')
ax1.set_ylabel('Residue Index')
plt.colorbar(im1, ax=ax1, label='Distance (Å)')

# 2. Binary Contact Map (AI/NMR Style)
# Threshold < 8 Angstroms is standard for CASP/AlphaFold evaluation
im2 = ax2.imshow(binary_map, cmap='Greys', interpolation='nearest')
ax2.set_title('Binary Contact Map (Threshold < 8Å)')
ax2.set_xlabel('Residue Index')
ax2.set_yticks([]) # Hide Y ticks for cleanliness

plt.tight_layout()
plt.show()

print("📊 INTERPRETATION:")
print("Dark patterns away from the diagonal indicate FOLDED structure.")
print("A Thick Diagonal with no off-diagonal spots = DISORDERED/UNFOLDED chain.")


### 📊 3. Chemical Shift Index (CSI)

The **Chemical Shift Index** is a method to visualize secondary structure directly from the raw data, without calculating a full 3D structure.

We plot the deviation of the Alpha-Carbon ($C_\alpha$) shift from its random coil value:
$$ \Delta \delta = \delta_{measured} - \delta_{random\_coil} $$

*   **Positive Clusters (> +0.7 ppm)**: Indicate **Alpha Helices**.
*   **Negative Clusters (< -0.7 ppm)**: Indicate **Beta Sheets**.
*   **Near Zero**: Indicates **Random Coil / Loops**.


In [ ]:
# Calculate CSI
import importlib
import synth_pdb.chemical_shifts
# Force reload to pick up the new function definition from disk
importlib.reload(synth_pdb.chemical_shifts)
from synth_pdb.chemical_shifts import calculate_csi

# 1. Get Deviations
csi_data = calculate_csi(shifts, structure)
chain_id = list(csi_data.keys())[0]
deltas = csi_data[chain_id]

res_nums = sorted(deltas.keys())
values = [deltas[r] for r in res_nums]

# 2. Plotting
plt.figure(figsize=(10, 4))
colors = ['red' if v > 0 else 'blue' for v in values]
plt.bar(res_nums, values, color=colors, alpha=0.7)

# 3. Add Threshold Lines (The "CSI" definition)
plt.axhline(0.7, color='black', linestyle='--', linewidth=1, label='Helix Threshold (+0.7)')
plt.axhline(-0.7, color='black', linestyle=':', linewidth=1, label='Sheet Threshold (-0.7)')
plt.axhline(0, color='black', linewidth=0.5)

plt.title(f'Chemical Shift Index (CSI) - $C_\\alpha$ Deviations')
plt.ylabel('$\\Delta \\delta$ (ppm)')
plt.xlabel('Residue Number')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.show()
